## Load

In [1]:
import json

In [2]:
with open("raw/ltl.txt", "r") as f:
    raw_ltls = f.read().splitlines()
    unique_ltls = set(raw_ltls)
with open("raw/eng.txt", 'r') as f:
    raw_engs = f.read().splitlines()

DPs = []
for ltl, eng in zip(raw_ltls, raw_engs):
    DPs.append({'ltl': ltl, 'eng': eng})

In [3]:
for ltl in sorted(unique_ltls, key=lambda x: len(x)):
    print(ltl)

G & U S ! C F C
G & U S ! A F A
G & U S ! Y F Y
G & U S ! R F R
G & U S ! B F B


In [5]:
print("Clean-up Domain, with augmentation")
print("Number of Data Points", len(DPs))
print("Number of unique LTL expressions:", len(unique_ltls))
print("Number of unique LTL structures:", 1)

Clean-up Domain, with augmentation
Number of Data Points 744
Number of unique LTL expressions: 5
Number of unique LTL structures: 1


In [6]:
seen = set()
from random import shuffle, seed
seed(0)
# shuffle(DPs)
for dp in DPs:
    if dp['ltl'] not in seen:
        seen.add(dp['ltl'])
        print(dp['ltl'])
        print(dp['eng'])
        print()

G & U S ! A F A
move the arm to the block and pick it up next placing it in the bin and coming back to pick up the next block

G & U S ! R F R
watch for blocks to be set down and then move them into the basket but not any orange ones

G & U S ! B F B
watch for blocks to be set down and then move them into the basket but not any blue ones

G & U S ! C F C
watch for blocks to be set down and then move them into the basket but not any green ones

G & U S ! Y F Y
watch for blocks to be set down and then move them into the basket but not any yellow ones



## Clean up LTL expressions 

In [7]:
APs = [
    {'ap': "A", 'eng': "any cubes"},
    {'ap': "R", 'eng': "any non red cubes"},
    {'ap': "B", 'eng': "any non blue cubes"},
    {'ap': "Y", 'eng': "any non yellow cubes"},
    {'ap': "C", 'eng': "any non green cubes"},
]

In [8]:
def build_type_A(ap1):
    return {
        'raw_ltl': f"G & U S ! {ap1['ap']} F {ap1['ap']}",
        'canonical_ltl': f"globally ( and ( until ( scan , not ( {ap1['eng']} ) ) , finally ( {ap1['eng']} ) ) )",
        'eng': f"Look for and pick up {ap1['eng']} and put them in crate."}

In [9]:
translation_seeds = []
for r1 in APs:
    translation_seeds.append(build_type_A(r1))

In [10]:
print(len(translation_seeds))

5


In [11]:
seed_ltls = set([t['raw_ltl'] for t in translation_seeds])
print(unique_ltls - seed_ltls)

set()


### Save the canonical decoding list

In [12]:
possible_decodings = {}
for seed in translation_seeds:
    canonical = seed['canonical_ltl']
    possible_decodings[canonical] = {
        'formula': canonical,
        'raw': seed['raw_ltl'],
    }

In [13]:
with open("canonical.json", 'w') as f:
    f.write(json.dumps(possible_decodings, indent=2))

### Save translation seed for zero shot

In [14]:
with open("train_seed.jsonl", "w") as f:
    for dp in translation_seeds:
        better_ltl = dp['canonical_ltl']
        entry = {'canonical': better_ltl, 'formula': better_ltl, 'natural': dp['eng']}
        json.dump(entry, f)
        f.write('\n')

### Save golden data for evaluation  

In [15]:
raw_canonical_mapping = {
    seed['raw_ltl']: seed['canonical_ltl'] for seed in translation_seeds
}

In [16]:
len(DPs)

744

In [17]:
with open("golden.jsonl", "w") as f:
    for dp in DPs:
        entry = {
            'canonical': raw_canonical_mapping[dp['ltl']],
            'formula': raw_canonical_mapping[dp['ltl']],
            'natural': dp['eng'],
            'raw_ltl': dp['ltl'],
        }
        json.dump(entry, f)
        f.write('\n')